In [ ]:
pip install simpletransformers

In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from simpletransformers.classification import ClassificationModel
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

In [3]:
# import data
# 'all_contents.csv' contain all input files but merged into one csv file
data = pd.read_csv('all_contents.csv')
data.head()

,file_path,content,type
0,../../data\business\business_1.txt,Lufthansa flies back to profit\n\nGerman airli...,business
1,../../data\business\business_10.txt,Winn-Dixie files for bankruptcy\n\nUS supermar...,business
2,../../data\business\business_100.txt,US economy still growing says Fed\n\nMost area...,business
3,../../data\business\business_11.txt,Saab to build Cadillacs in Sweden\n\nGeneral M...,business
4,../../data\business\business_12.txt,Bank voted 8-1 for no rate change\n\nThe decis...,business


In [5]:
type_substitution = {
    'business':0,
    'entertainment':1,
    'food':2,
    'graphics':3,
    'historical':4,
    'medical':5,
    'politics':6,
    'space':7,
    'sport':8,
    'technology':9
}

str_labels = list(type_substitution.keys())
integer_labels = list(type_substitution.values())

print(str_labels)
print(integer_labels)

data['type'] = data['type'].replace(str_labels, integer_labels)

['business', 'entertainment', 'food', 'graphics', 'historical', 'medical', 'politics', 'space', 'sport', 'technology']
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


In [6]:
data.shape

(1000, 3)

In [7]:
list(data['content'])[1]

'Winn-Dixie files for bankruptcy\n\nUS supermarket group Winn-Dixie has filed for bankruptcy protection after succumbing to stiff competition in a market dominated by Wal-Mart.\n\nWinn-Dixie, once among the most profitable of US grocers, said Chapter 11 protection would enable it to successfully restructure. It said its 920 stores would remain open, but analysts said it would most likely off-load a number of sites. The Jacksonville, Florida-based firm has total debts of $1.87bn (£980m). In its bankruptcy petition it listed its biggest creditor as US foods giant Kraft Foods, which it owes $15.1m.\n\nAnalysts say Winn-Dixie had not kept up with consumers\' demands and had also been burdened by a number of stores in need of upgrading. A 10-month restructuring plan was deemed a failure, and following a larger-than-expected quarterly loss earlier this month, Winn-Dixie\'s slide into bankruptcy was widely expected. The company\'s new chief executive Peter Lynch said Winn-Dixie would use the 

In [8]:
X = data['content']
y = data['type']

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.25, random_state=42, stratify = y)

df_for_training = pd.concat([X_train, y_train], axis=1)
df_for_training.head()

,content,type
728,Are you people posting this to sci.space becau...,7
219,1 pound medium russet potatoes\n ...,2
414,World War I at Sea\nIn the years before World ...,4
332,In <1pscti$aqe@travis.csd.harris.com> srp@trav...,3
7,China keeps tight rein on credit\n\nChina's ef...,0


In [11]:
df_for_training.shape

(750, 2)

In [12]:
from simpletransformers.classification import ClassificationModel

# training params
train_args = {
    "reprocess_input_data":True,
    "fp16":False,
    "num_train_epochs":2
}

# model
model = ClassificationModel("roberta", "roberta-base", use_cuda = False, args=train_args, num_labels = 10)
# train the model
model.train_model(df_for_training)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

...

  0%|          | 0/750 [00:00<?, ?it/s]

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Running Epoch 0 of 2:   0%|          | 0/94 [00:00<?, ?it/s]

Running Epoch 1 of 2:   0%|          | 0/94 [00:00<?, ?it/s]

(188, 0.6426436854625478)

In [13]:
import pickle
output_path = "roberta_dcm.pkl"
with open(output_path, mode = 'wb') as file:
  pickle.dump(model, file)

In [14]:
# make predictions
predictions, raw_outputs = model.predict(list(X_test))

  0%|          | 0/250 [00:00<?, ?it/s]

  0%|          | 0/32 [00:00<?, ?it/s]

In [15]:
# evalute model accuracy
y_test_numpy = np.array(y_test)
print("accuracy: ", accuracy_score(predictions, y_test_numpy))

accuracy:  0.96


In [21]:
basketball = 0
with open("basketball.txt", mode = "r") as f:
  basketball = f.read()

basketball = [basketball]
single_prediction, raw_output = model.predict(basketball)
print(single_prediction)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

[8]
